In [4]:
import sys
import re 
import numpy as np 
import pandas as pd
import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
import tensorflow as tf
import play
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Bidirectional, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from music21 import converter, instrument, note, chord, stream
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
songs = glob('Rap/*.mid')

In [6]:
songs = songs[::]

In [7]:
def get_notes():
    print("Obtendo notas")
    notes = []
    for file in songs:
        # converting .mid file to stream object
        midi = converter.parse(file)
        notes_to_parse = []
        try:
            # Given a single stream, partition into a part for each unique instrument
            parts = instrument.partitionByInstrument(midi)
        except:
            pass
        if parts: # if parts has instrument parts 
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes
    
        for element in notes_to_parse: 
            if isinstance(element, note.Note):
                # if element is a note, extract pitch
                notes.append(str(element.pitch))
            elif(isinstance(element, chord.Chord)):
                # if element is a chord, append the normal form of the 
                # chord (a list of integers) to the list of notes. 
                notes.append('.'.join(str(n) for n in element.normalOrder))
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    return notes

In [8]:
def prepare_sequences(notes, n_vocab): 
    sequence_length = 100

    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i: i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)
    network_output = tf.keras.utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [9]:
def create_network(network_in, n_vocab): 
    """Create the model architecture"""
    model = Sequential()
    model.add(LSTM(128, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

In [11]:
def train(model, network_input, network_output, epochs): 
    """
    Train the neural network
    """
    # Create checkpoint to save the best model weights.
    filepath = './modelo_teste.h5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
    history = model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint])

In [12]:
def train_network():
    """
    Obter notas
    Gerar sequências de entrada e saída
    Criar um modelo
    Treinar o modelo para as épocas fornecidas
    """
    epochs = 50

    notes = get_notes()
    print('Notas processadas')

    n_vocab = len(set(notes))
    print('Vocabulário gerado')

    network_in, network_out = prepare_sequences(notes, n_vocab)
    print('Entrada e Saída processadas')

    model = create_network(network_in, n_vocab)
    print('Modelo criado')
    print('Treinamento em andamento')
    
    train(model, network_in, network_out, epochs)
    print('Treinamento concluído')
    return model

In [13]:
### Train the model 
model = train_network()
model.save_weights('trained_model.h5')

Obtendo notas


C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\music21\midi\translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'By Nokio-Copyrighted \xae'>; getting generic Instrument
  warnings.warn(
C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\music21\midi\translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'NoKiO \xa9 2000'>; getting generic Instrument
  warnings.warn(


Notas processadas
Vocabulário gerado
Entrada e Saída processadas


Modelo criado
Treinamento em andamento
Epoch 1/50

323/323 [==============================] - 39s 111ms/step - loss: 4.5727
Epoch 2/50
  1/323 [..............................] - ETA: 35s - loss: 4.4280

C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


323/323 [==============================] - 36s 111ms/step - loss: 4.2341
Epoch 3/50
323/323 [==============================] - 35s 108ms/step - loss: 3.9658
Epoch 4/50
323/323 [==============================] - 36s 110ms/step - loss: 3.7291
Epoch 5/50
323/323 [==============================] - 34s 105ms/step - loss: 3.4824
Epoch 6/50
323/323 [==============================] - 34s 106ms/step - loss: 3.1598
Epoch 7/50
323/323 [==============================] - 34s 105ms/step - loss: 2.9307
Epoch 8/50
323/323 [==============================] - 34s 104ms/step - loss: 2.7272
Epoch 9/50
323/323 [==============================] - 33s 103ms/step - loss: 2.5486
Epoch 10/50
323/323 [==============================] - 33s 103ms/step - loss: 2.3245
Epoch 11/50
323/323 [==============================] - 34s 104ms/step - loss: 2.0788
Epoch 12/50
323/323 [==============================] - 35s 108ms/step - loss: 1.8197
Epoch 13/50
323/323 [==============================] - 35s 110ms/step - loss: 1.5420

In [14]:
def get_inputSequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
    
    network_input = np.reshape(network_input, (len(network_input), 100, 1))
    
    return (network_input)

In [15]:
def generate_notes(model, network_input, pitchnames, n_vocab, temperature=2.0):
    """ Generate notes from the neural network based on a sequence of notes """
    # Pick a random integer
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # pick a random sequence from the input as a starting point for the prediction
    pattern = list(network_input[start])
    prediction_output = []
    
    print('Generating notes........')

    # generate 500 notes
    for note_index in range(300):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        pattern = np.array(pattern)
        prediction_input = prediction_input / float(n_vocab)

        # Ajuste da temperatura
        prediction = model.predict(prediction_input, verbose=0)[0]
        prediction = np.log(prediction) / temperature
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)

        # Escolha aleatória com base nas probabilidades ajustadas
        index = np.random.choice(len(prediction), p=prediction)
        result = int_to_note[index]

        # Storing the predicted output
        prediction_output.append(result)

        pattern = np.append(pattern, index)
        pattern = pattern[1:]

    print('Notes Generated...')
    print(prediction_output)
    return prediction_output

In [16]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    
    print('Saving Output file as midi....')

    midi_stream.write('midi', fp='test_output4.mid')

In [17]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))
    
    print('Initiating music generation process.......')
    
    network_input = get_inputSequences(notes, pitchnames, n_vocab)
    normalized_input = network_input / float(n_vocab)
    model = create_network(normalized_input, n_vocab)
    print('Loading Model weights.....')
    model.load_weights('./modelo_teste.h5')
    print('Model Loaded')
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [18]:

generate()

Initiating music generation process.......
Loading Model weights.....
Model Loaded
Generating notes........


C:\Users\PC\AppData\Local\Temp\ipykernel_2148\1127633650.py:22: RuntimeWarning: divide by zero encountered in log
  prediction = np.log(prediction) / temperature


Notes Generated...
['C6', 'F5', '5.9.0', 'C6', 'A5', 'F5', 'C5', 'A4', 'F4', '5.9.0', 'A4', 'C5', 'A4', '0.3.5', 'C5', 'F5', 'F5', '10.2.5', 'D6', 'B-5', 'F5', 'D5', 'B-4', 'F4', '10.2.5', 'B-4', 'D5', 'B-4', '2.5', 'D5', 'F5', 'F5', '5.9.0', 'F6', 'C6', 'A5', 'F5', 'C5', 'A4', '5.9', 'F4', 'C4', 'F4', '9.0', 'A4', 'C5', 'C5', '9.0', 'F5', 'F5', 'D5', '10.0.2', 'G5', 'G5', 'C5', '9.0', 'F5', 'F5', 'C5', '7.10.0', 'E5', 'E5', 'C5', '6.9.0', 'E-5', 'E-5', 'C5', '6.9.0', 'D5', 'D5', '6.9', '6.9.0', 'E-5', 'E-5', '6.9', '6.9.0', 'D5', 'D5', '5.10', '5.10', 'E-5', 'E-5', '5.10', '5.10', 'D5', 'D5', '5.9', '5.9.0', 'C5', 'C5', '4.7', '7.10.0', 'C5', 'C5', '5.9.0', 'F3', 'C4', 'A3', 'F3', 'C3', 'A2', '5.9.0', 'F2', 'A2', 'C3', '0.3.5', 'A2', 'C3', 'F3', '10.2.5', 'F3', 'D4', 'B-3', 'F3', 'F3', 'B-2', '10.2.5', 'F2', 'B-2', 'D3', '2.5', 'B-2', 'D3', 'F3', '5.9.0', 'F3', 'F4', 'C4', 'A3', 'F3', 'C3', '5.9.0', 'A2', 'F2', 'C2', '5.9.0', 'F2', 'A2', 'C3', 'C6', '9.0', 'F6', 'F6', 'D6', '10.0.2', 

In [21]:
play.play_midi('test_output4.mid')

Music file test_output4.mid loaded!
